In [1]:
from manim import *

Manim Community v0.18.0

In [2]:
config.media_width = "75%"
config.verbosity = "WARNING"
config.background_color = "#24043d"

In [3]:
import cuadro_datos as cd
import herramientas as h

In [4]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):
    
        def crear_snippet(mi_codigo):
            """Recibe un objeto Code y lo dispone en pantalla letra por letra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/8
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def generar_snippet(mi_codigo):
            """Versión generador de crear_snippet"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i]) / 7.5
                # Animar otros objetos aquí, según sea necesario
                yield self.play(FadeIn(mi_codigo[1][i]))
                yield self.play(AddTextLetterByLetter(mi_codigo[2][i], run_time=chars_por_seg, rate_func=linear))
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):
            """Crea una secuencia de figuras que sirven de contenedores para un texto.
            Hacemos uso de arrange_in_grid para que el texto quede contenido en cada una de las figuras."""
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace").scale(0.85)
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        #grilla = h.GrillaRegla()
        #self.add(grilla)

        ##### Memoria
        #################### Crear la grilla para simular una memoria####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)
            circle.set_fill(BLACK, opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)

        self.wait(3)
        #### Fin memoria
        
        ## Animación
        mem_heap1 = VGroup(*[grilla[0][i] for i in range(12, 17)])
        explicacion = Text("Crear una lista con 5 elementos", font = "Arial Black").scale(0.7).shift(3.5*UP)
        resultado = Text("Resultado:", font = "Arial Black").scale(0.6).shift(4.5*LEFT + 0.3 * UP)
        mov = 2.25 * UP + 4.5 * LEFT
        codigo = crear_codigo("""lista = [1, 2, 3, 4, 5]
print(lista)""", mov)
        texto_interno = Text("12345", font="Arial Black")
        # poner en escena la explicación
        self.play(Write(explicacion))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # texto resultado
        self.play(Write(resultado))
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo)
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, None, 0.8)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # una lista que queda tras de escenas
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy().move_to(4.5 * LEFT + 0.4*DOWN)
        
        # cuadrado para contener el nombre de variable "lista"
        rect_lista = Rectangle(width= 1.8, height = 0.9, color="#CE482A", fill_color="#CE482A", fill_opacity=1).scale(0.6)
        var_lista = Text("'lista'").scale(0.5).move_to(rect_lista)
        stack_lista = Group(*[rect_lista, var_lista])
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        pointer = Arrow(codigo[2][0].get_right() + 1.25 * RIGHT, codigo[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        self.play(GrowArrow(pointer))
        # mover nombre de variable "lista" a la stack
        self.play(stack_lista.animate.move_to(grilla[0][10].get_right()))
        
        # copia de la lista a memoria
        lista_heap = lista.copy().scale(0.667)  
        self.play(lista_heap.animate.move_to(mem_heap1))
        
        # copio lista_heap ya posicionadio en la heap
        lh_copia = lista_heap.copy()
        self.add(lh_copia)
        # animo la segunda línea de Code()
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer2 = Arrow(codigo[2][1].get_right() + 1.25 * RIGHT, codigo[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer2))
        # animación de llamado a variable y referencia a objeto en memoria
        lp = DashedLine(pointer2.get_right(), rect_lista.get_left(), color=YELLOW)
        self.play(Create(lp), run_time=1.25)
        self.play(Uncreate(lp), run_time=1.25)
        pointer_lista = Arrow(rect_lista.get_bottom(), lh_copia.get_center(), color = "#CE482A")
        self.play(GrowArrow(pointer_lista))
        # indicar objeto u objetos referenciados
        for i in range(2):
            self.play(Circumscribe(lh_copia), color = "#CE482A")
        self.play(FadeOut(pointer_lista))
        # animar creación de objeto en pantalla/consola
        self.play(Transform(lh_copia, lista_copia))
        self.wait(2)
        # limpiar la pantalla
        self.play(FadeOut(lh_copia), FadeOut(codigo), FadeOut(explicacion), FadeOut(pointer))
        self.wait(2)

        # Segunda animación
        explicacion2 = Text("Obtener el primer elemento de una lista", font = "Arial Black").scale(0.7).shift(3.5*UP)
        codigo2 = crear_codigo("""lista = [1, 2, 3, 4, 5]
primero = lista[0]
print(lista[0])""", mov)

        texto_interno = Text("12345", font="Arial Black")
        # poner en escena la explicación
        self.play(Write(explicacion2))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo2[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo2)

        # copio el primero de la lista de resultado
        lista_copia_primero = Group(lista_copia[0][0], lista_copia[1][0])
        lc_primero_copia = lista_copia_primero
        # copio lista_heap ya posicionadio en la heap
        lh_primero = Group(*[lista_heap[0][0], lista_heap[1][0]])
        lh_primero_copia = lh_primero.copy()
        self.add(lh_primero_copia)
        
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
            
        pointer3 = Arrow(codigo2[2][0].get_right() + 1.25 * RIGHT, codigo2[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=RED)
        self.play(GrowArrow(pointer3))
                           

        # animo la segunda línea de Code() (primero = lista[0])
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
                            
        # objetos para animar creación nombre de variable
        rect_primero = Rectangle(width= 1.8, height = 0.9, color=BLUE, fill_color=BLUE, fill_opacity=1).scale(0.6)
        var_primero = Text("'primero'").scale(0.425).move_to(rect_primero)
        stack_primero = Group(*[rect_primero, var_primero])
        # puntero de línea de la declaración de la variable primero
        pointer4 = Arrow(codigo2[2][1].get_right() + 1.25 * RIGHT, codigo2[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        # animación del puntero anterior
        self.play(Transform(pointer3, pointer4))
        # mover nombre de variable "lista" a la stack
        self.play(stack_primero.animate.move_to(grilla[0][0].get_right()))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer5 = Arrow(codigo2[2][2].get_right() + 1.25 * RIGHT, codigo2[2][2].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        self.play(Transform(pointer3, pointer5))             
        
        # animación de llamado a variable y referencia a objeto en memoria
        lp = DashedLine(pointer5.get_right(), rect_primero.get_left(), color=YELLOW)
        self.play(Create(lp), run_time=1.25)
        self.play(Uncreate(lp), run_time=1.25)
        # pointer de referencia a objeto en memoria
        pointer_primero = Arrow(rect_primero.get_bottom(), lh_primero_copia.get_center(), color = BLUE)
        self.play(GrowArrow(pointer_primero))
        # indicar el o los objetos referenciados
        for i in range(2):
            self.play(Circumscribe(lh_primero_copia), color = "#3333FF")
        # limpiar la escena de objetos innecesarios
        self.play(FadeOut(pointer_primero))
        self.play(Transform(lh_primero_copia, lc_primero_copia))
        self.wait(2)
        self.play(FadeOut(stack_primero), FadeOut(codigo2), FadeOut(explicacion2), FadeOut(pointer3), FadeOut(lh_primero_copia))
        self.wait(2)

In [91]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):
    
        def crear_snippet(mi_codigo):
            """Recibe un objeto Code y lo dispone en pantalla letra por letra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/8
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def generar_snippet(mi_codigo):
            """Versión generador de crear_snippet"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i]) / 7.5
                # Animar otros objetos aquí, según sea necesario
                yield self.play(FadeIn(mi_codigo[1][i]))
                yield self.play(AddTextLetterByLetter(mi_codigo[2][i], run_time=chars_por_seg, rate_func=linear))
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):
            """Crea una secuencia de figuras que sirven de contenedores para un texto.
            Hacemos uso de arrange_in_grid para que el texto quede contenido en cada una de las figuras."""
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace").scale(0.85)
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ##### Memoria
        #################### Crear la grilla para simular una memoria####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)
            circle.set_fill(BLACK, opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)

        self.wait(3)
        #### Fin memoria
        
        ## Posicionar nombre de variable y dato en memoria
        mem_heap1 = VGroup(*[grilla[0][i] for i in range(12, 17)])
        resultado = Text("Resultado:", font = "Arial Black").scale(0.6).shift(4.5*LEFT + 0.3 * UP)
        mov = 2.25 * UP + 4.5 * LEFT
        texto_interno = Text("12345", font="Arial Black")
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # una lista que queda tras de escenas
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy().move_to(8.1 * LEFT + 0.4*DOWN) 
        # cuadrado para contener el nombre de variable "lista"
        rect_lista = Rectangle(width= 1.8, height = 0.9, color="#CE482A", fill_color="#CE482A", fill_opacity=1).scale(0.6)
        var_lista = Text("'lista'").scale(0.5).move_to(rect_lista)
        stack_lista = Group(*[rect_lista, var_lista])
        
        # mover nombre de variable "lista" a la stack
        stack_lista.move_to(grilla[0][10].get_right())
        self.add(stack_lista)
        
        # copia de la lista a memoria
        lista_heap = lista.copy().scale(0.667)  
        lista_heap.move_to(mem_heap1)
        self.add(lista_heap)

        # Segunda animación
        explicacion2 = Text("Obtener el último elemento de una lista", font = "Arial Black").scale(0.7).shift(3.5*UP)
        
        codigo2 = crear_codigo("""lista = [1, 2, 3, 4, 5]
ultimo = lista[-1]
print(ultimo)""", mov)

        # poner en escena la explicación
        self.play(Write(explicacion2))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo2[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # texto resultado
        self.play(Write(resultado))
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo2)

        # copio el último de la lista de resultado
        lista_copia_ultimo = Group(lista_copia[0][-1], lista_copia[1][-1])
        lc_ultimo_copia = lista_copia_ultimo
        # copio lista_heap ya posicionadio en la heap
        lh_ultimo = Group(*[lista_heap[0][-1], lista_heap[1][-1]])
        lh_ultimo_copia = lh_ultimo.copy()
        self.add(lh_ultimo_copia)
        
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
            
        pointer3 = Arrow(codigo2[2][0].get_right() + 1.25 * RIGHT, codigo2[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=RED)
        self.play(GrowArrow(pointer3))
                           

        # animo la segunda línea de Code() (primero = lista[0])
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
                            
        # objetos para animar creación nombre de variable
        rect_ultimo = Rectangle(width= 1.8, height = 0.9, color=BLUE, fill_color=BLUE, fill_opacity=1).scale(0.6)
        var_ultimo = Text("'último'").scale(0.425).move_to(rect_ultimo)
        stack_ultimo = Group(*[rect_ultimo, var_ultimo])
        # puntero de línea de la declaración de la variable "ultimo"
        pointer4 = Arrow(codigo2[2][1].get_right() + 1.25 * RIGHT, codigo2[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        # animación del puntero anterior
        self.play(Transform(pointer3, pointer4))
        # mover nombre de variable "lista" a la stack
        self.play(stack_ultimo.animate.move_to(grilla[0][0].get_right()))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer5 = Arrow(codigo2[2][2].get_right() + 1.25 * RIGHT, codigo2[2][2].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        self.play(Transform(pointer3, pointer5))             
        
        # animación de llamado a variable y referencia a objeto en memoria
        lp = DashedLine(pointer5.get_right(), rect_ultimo.get_left(), color=YELLOW)
        self.play(Create(lp), run_time=1.25)
        self.play(Uncreate(lp), run_time=1.25)
        # pointer de referencia a objeto en memoria
        pointer_ultimo = Arrow(rect_ultimo.get_bottom(), lh_ultimo_copia.get_center(), color = BLUE)
        self.play(GrowArrow(pointer_ultimo))
        # indicar el o los objetos referenciados
        for i in range(2):
            self.play(Circumscribe(lh_ultimo_copia), color = "#3333FF")
        # limpiar la escena de objetos innecesarios
        self.play(FadeOut(pointer_ultimo))
        self.play(Transform(lh_ultimo_copia, lc_ultimo_copia))
        self.wait(2)
        self.play(FadeOut(stack_ultimo), FadeOut(codigo2), FadeOut(explicacion2), FadeOut(pointer3), FadeOut(lh_ultimo_copia))
        self.wait(2)

In [92]:
lista = [1, 2, 3, 4, 5]
lista[1:4]

[2, 3, 4]

In [94]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):
    
        def crear_snippet(mi_codigo):
            """Recibe un objeto Code y lo dispone en pantalla letra por letra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/8
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def generar_snippet(mi_codigo):
            """Versión generador de crear_snippet"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i]) / 7.5
                # Animar otros objetos aquí, según sea necesario
                yield self.play(FadeIn(mi_codigo[1][i]))
                yield self.play(AddTextLetterByLetter(mi_codigo[2][i], run_time=chars_por_seg, rate_func=linear))
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):
            """Crea una secuencia de figuras que sirven de contenedores para un texto.
            Hacemos uso de arrange_in_grid para que el texto quede contenido en cada una de las figuras."""
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace").scale(0.85)
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ##### Memoria
        #################### Crear la grilla para simular una memoria####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)
            circle.set_fill(BLACK, opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)

        self.wait(3)
        #### Fin memoria
        
        ## Posicionar nombre de variable y dato en memoria
        mem_heap1 = VGroup(*[grilla[0][i] for i in range(12, 17)])
        resultado = Text("Resultado:", font = "Arial Black").scale(0.6).shift(4.5*LEFT + 0.3 * UP)
        mov = 2.25 * UP + 4.5 * LEFT
        texto_interno = Text("12345", font="Arial Black")
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # una lista que queda tras de escenas
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy().move_to(5.4 * LEFT + 0.4*DOWN) 
        # cuadrado para contener el nombre de variable "lista"
        rect_lista = Rectangle(width= 1.8, height = 0.9, color="#CE482A", fill_color="#CE482A", fill_opacity=1).scale(0.6)
        var_lista = Text("'lista'").scale(0.5).move_to(rect_lista)
        stack_lista = Group(*[rect_lista, var_lista])
        
        # mover nombre de variable "lista" a la stack
        stack_lista.move_to(grilla[0][10].get_right())
        self.add(stack_lista)
        
        # copia de la lista a memoria
        lista_heap = lista.copy().scale(0.667)  
        lista_heap.move_to(mem_heap1)
        self.add(lista_heap)

        # Segunda animación
        explicacion2 = Text("Obtener una porción de la lista", font = "Arial Black").scale(0.7).shift(3.5*UP)
        
        codigo2 = crear_codigo("""lista = [1, 2, 3, 4, 5]
internos = lista[1:4]
print(internos)""", mov)

        # poner en escena la explicación
        self.play(Write(explicacion2))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo2[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # texto resultado
        self.play(Write(resultado))
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo2)

        # copio el último de la lista de resultado
        lista_copia_internos = Group(lista_copia[0][1:4], lista_copia[1][1:4])
        lc_internos_copia = lista_copia_internos
        # copio lista_heap ya posicionadio en la heap
        lh_internos = Group(*[lista_heap[0][1:4], lista_heap[1][1:4]])
        lh_internos_copia = lh_internos.copy()
        self.add(lh_internos_copia)
        
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
            
        pointer3 = Arrow(codigo2[2][0].get_right() + 1.25 * RIGHT, codigo2[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=RED)
        self.play(GrowArrow(pointer3))
                           
        # animo la segunda línea de Code() (primero = lista[0])
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
                            
        # objetos para animar creación nombre de variable
        rect_internos = Rectangle(width= 1.8, height = 0.9, color=BLUE, fill_color=BLUE, fill_opacity=1).scale(0.6)
        var_internos = Text("'internos'").scale(0.375).move_to(rect_internos)
        stack_internos = Group(*[rect_internos, var_internos])
        # puntero de línea de la declaración de la variable "internos"
        pointer4 = Arrow(codigo2[2][1].get_right() + 1.25 * RIGHT, codigo2[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        # animación del puntero anterior
        self.play(Transform(pointer3, pointer4))
        # mover nombre de variable "lista" a la stack
        self.play(stack_internos.animate.move_to(grilla[0][0].get_right()))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer5 = Arrow(codigo2[2][2].get_right() + 1.25 * RIGHT, codigo2[2][2].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        self.play(Transform(pointer3, pointer5))             
        
        # animación de llamado a variable y referencia a objeto en memoria
        lp = DashedLine(pointer5.get_right(), rect_internos.get_left(), color=YELLOW)
        self.play(Create(lp), run_time=1.25)
        self.play(Uncreate(lp), run_time=1.25)
        # pointer de referencia a objeto en memoria
        pointer_internos = Arrow(rect_internos.get_bottom(), lh_internos_copia.get_center(), color = BLUE)
        self.play(GrowArrow(pointer_internos))
        # indicar el o los objetos referenciados
        for i in range(2):
            self.play(Circumscribe(lh_internos_copia), color = "#3333FF")
        # limpiar la escena de objetos innecesarios
        self.play(FadeOut(pointer_internos))
        self.play(Transform(lh_internos_copia, lc_internos_copia))
        self.wait(2)
        self.play(FadeOut(stack_internos), FadeOut(codigo2), FadeOut(explicacion2), FadeOut(pointer3), FadeOut(lh_internos_copia))
        self.wait(2)

In [103]:
lista = [1, 2, 3, 4, 5]
id(lista)

2031695965056

In [108]:
letra = 'choclo'
print(id(letra))
lista.append(letra)
#lista2 = ["Python", "3", 6]
#lista.append(lista2)
print(lista)

2031694794160
[1, 2, 3, 4, 5, 'a', 'a', 'choclo']


In [109]:
print(id(lista))
print(id(letra))

2031695965056
2031694794160


In [110]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):
    
        def crear_snippet(mi_codigo):
            """Recibe un objeto Code y lo dispone en pantalla letra por letra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/8
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def generar_snippet(mi_codigo):
            """Versión generador de crear_snippet"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i]) / 7.5
                # Animar otros objetos aquí, según sea necesario
                yield self.play(FadeIn(mi_codigo[1][i]))
                yield self.play(AddTextLetterByLetter(mi_codigo[2][i], run_time=chars_por_seg, rate_func=linear))
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):
            """Crea una secuencia de figuras que sirven de contenedores para un texto.
            Hacemos uso de arrange_in_grid para que el texto quede contenido en cada una de las figuras."""
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace").scale(0.85)
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ##### Memoria
        #################### Crear la grilla para simular una memoria####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)
            circle.set_fill(BLACK, opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)

        self.wait(3)
        #### Fin memoria
        
        ## Posicionar nombre de variable y dato en memoria
        mem_heap1 = VGroup(*[grilla[0][i] for i in range(12, 17)])
        resultado = Text("Resultado:", font = "Arial Black").scale(0.6).shift(4.5*LEFT + 0.3 * UP)
        mov = 2.25 * UP + 4.5 * LEFT
        texto_interno = Text("12345", font="Arial Black")
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # una lista que queda tras de escenas
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy().move_to(5.4 * LEFT + 0.4*DOWN) 
        # cuadrado para contener el nombre de variable "lista"
        rect_lista = Rectangle(width= 1.8, height = 0.9, color="#CE482A", fill_color="#CE482A", fill_opacity=1).scale(0.6)
        var_lista = Text("'lista'").scale(0.5).move_to(rect_lista)
        stack_lista = Group(*[rect_lista, var_lista])
        
        # mover nombre de variable "lista" a la stack
        stack_lista.move_to(grilla[0][10].get_right())
        self.add(stack_lista)
        
        # copia de la lista a memoria
        lista_heap = lista.copy().scale(0.667)  
        lista_heap.move_to(mem_heap1)
        self.add(lista_heap)

        # Segunda animación
        explicacion2 = Text("Agregar un elemento a una lista", font = "Arial Black").scale(0.7).shift(3.5*UP)
        
        codigo2 = crear_codigo("""lista = [1, 2, 3, 4, 5]
letra = 'a'
lista.append(letra)
print(lista)""", mov)

        # poner en escena la explicación
        self.play(Write(explicacion2))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo2[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # texto resultado
        self.play(Write(resultado))
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo2)
        
        # copio el último de la lista de resultado
        lista_copia_internos = Group(lista_copia[0][1:4], lista_copia[1][1:4])
        lc_internos_copia = lista_copia_internos
        # copio lista_heap ya posicionadio en la heap
        lh_internos = Group(*[lista_heap[0][1:4], lista_heap[1][1:4]])
        lh_internos_copia = lh_internos.copy()
        self.add(lh_internos_copia)
        
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
            
        pointer3 = Arrow(codigo2[2][0].get_right() + 1.25 * RIGHT, codigo2[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=RED)
        self.play(GrowArrow(pointer3))
                           
        # animo la segunda línea de Code() (primero = lista[0])
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
                            
        # objetos para animar creación nombre de variable
        rect_internos = Rectangle(width= 1.8, height = 0.9, color=BLUE, fill_color=BLUE, fill_opacity=1).scale(0.6)
        var_internos = Text("'internos'").scale(0.375).move_to(rect_internos)
        stack_internos = Group(*[rect_internos, var_internos])
        # puntero de línea de la declaración de la variable "internos"
        pointer4 = Arrow(codigo2[2][1].get_right() + 1.25 * RIGHT, codigo2[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        # animación del puntero anterior
        self.play(Transform(pointer3, pointer4))
        # mover nombre de variable "lista" a la stack
        self.play(stack_internos.animate.move_to(grilla[0][0].get_right()))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer5 = Arrow(codigo2[2][2].get_right() + 1.25 * RIGHT, codigo2[2][2].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN) 
        self.play(Transform(pointer3, pointer5))             
        
        # animación de llamado a variable y referencia a objeto en memoria
        lp = DashedLine(pointer5.get_right(), rect_internos.get_left(), color=YELLOW)
        self.play(Create(lp), run_time=1.25)
        self.play(Uncreate(lp), run_time=1.25)
        # pointer de referencia a objeto en memoria
        pointer_internos = Arrow(rect_internos.get_bottom(), lh_internos_copia.get_center(), color = BLUE)
        self.play(GrowArrow(pointer_internos))
        # indicar el o los objetos referenciados
        for i in range(2):
            self.play(Circumscribe(lh_internos_copia), color = "#3333FF")
        # limpiar la escena de objetos innecesarios
        self.play(FadeOut(pointer_internos))
        self.play(Transform(lh_internos_copia, lc_internos_copia))
        self.wait(2)
        self.play(FadeOut(stack_internos), FadeOut(codigo2), FadeOut(explicacion2), FadeOut(pointer3), FadeOut(lh_internos_copia))
        self.wait(2)

In [18]:
def mi_generador():
    yield "Hola"
    yield "Mundo"
    yield "!"

# Crear una instancia del generador
gen = mi_generador()

# Generar el primer valor del generador
primer_valor = next(gen)
print(primer_valor)
print("ese fue un Hola")
next(gen)

Hola
ese fue un Hola


'Mundo'

In [279]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):

        def crear_snippet(mi_codigo):
            """Crea una ventana de código y lo muestra palabra por palabra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/7.5
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):      
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace")
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ##### Memoria
        #################### Crear la grilla####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)  #, buff=MED_LARGE_BUFF)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)#("#4D4D55", opacity =1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)#("#DEDDDD")
            circle.set_fill(BLACK, opacity=1)#("#DEDDDD", opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)
        # 

        self.wait(3)

        #### Fin memoria
        
        ## Animación
        explicacion = Text("Crear una lista con 5 elementos", font = "Arial Black").scale(0.7).shift(3.5*UP)
        mov = 2 * UP + 4 * LEFT
        codigo = crear_codigo("""lista = [1, 2, 3, 4, 5]
print(lista)""", mov)
        texto_interno = Text("12345", font="Arial Black")
        # poner en escena la explicación
        self.play(Write(explicacion))
        # agregar ventana de código (solo la ventana)
        self.add(codigo[0])
        # Al crear la snippet se crea la animación de las líneas de código
        crear_snippet(codigo)
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8).move_to(4.5 * LEFT * DOWN)
        # contenedores para tapar a los contenedores de las escena cuando necesite mostrar solo algunos elementos de la lista
        #telones = crear_contenedores(texto_interno, "#24043d", "#24043d", 0.8,  0).move_to(2*DOWN + 1.5 * LEFT).shift(1*DOWN)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # una lista que queda tras de escenas y una copia que formará parte de la escena
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy()
        #
        cuad_lista = Square(side_length=0.9, color="#CE482A", fill_color="#CE482A", fill_opacity=1).scale(0.6)
        #
        var_lista = Text("lista").scale(0.4).move_to(cuad_lista)
        stack_lista = Group(*[cuad_lista, var_lista])
        self.play(stack_lista.animate.move_to(grilla[0][0]))
        #

        # copia de la lista a memoria
        lista_heap = lista.copy().scale(0.667)
        lh_copia = lista_heap.copy()
        # subgrupo de la grilla al cual irá el dato lista
        mem_heap1 = VGroup(*[grilla[0][i] for i in range(12, 17)])
        self.play(lista_heap.animate.move_to(mem_heap1))

        self.play(Transform(lh_copia, lista_copia))
        
        # crear un telon rectangular para cubrir la lista original
        #telon = Rectangle(width=4.6, height=1.1, fill_color="#24043d", fill_opacity=1, stroke_width=0).move_to(lista)
        #self.add(telon)
        # primer elemento
        explicacion2 = Text("Obtener primer elemento", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion2))
        mi_texto2 = """lista = [1, 2, 3, 4, 5]
print(lista[0])"""
  
        codigo2 = crear_codigo(mi_texto2, mov)
        self.add(codigo2[0])
        crear_snippet(codigo2)

        elem0_copia = lista_heap[0].copy()
        lista0_copia = lista[0].copy()

        self.play(Transform(elem0_copia, lista0_copia))
        # Indicar
        #for i in range(2):
        #    self.play(Circumscribe(contenedores[0]))

        #self.play(telones[1].animate.move_to(contenedores[1]), telones[2].animate.move_to(contenedores[2]),
         #         telones[3].animate.move_to(contenedores[3]), telones[4].animate.move_to(contenedores[4]))

        # último elemento
        explicacion3 = Text("Obtener el último elemento", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion3))
        mi_texto3 = """lista = [1, 2, 3, 4, 5]
print(lista[-1])"""
  
        codigo3 = crear_codigo(mi_texto3, mov)
        self.add(codigo3[0])
        crear_snippet(codigo3)
        # Indicar
        for i in range(2):
            self.play(Circumscribe(contenedores[-1]))

        #self.play(telones[0].animate.move_to(contenedores[0]))
        #self.play(telones[-1].animate.shift(2 * DOWN))
        # Añadir elemento suelto
        explicacion4 = Text("Agregar un elementos", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion3))
        #self.play(FadeOut(telones))
        # append
        mi_texto4 = """lista = [1, 2, 3, 4, 5]
letra = 'a'
palabra = 'Python'
lista.append(letra)
lista.append(palabra)
print(lista)
"""
        letra = Text("'a'").scale(0.6)
        palabra = Text("'Python'").scale(0.3)
        #mov = 2 * UP
        codigo4 = crear_codigo(mi_texto4, mov)
        self.add(codigo4[0])
        crear_snippet(codigo4)
        
        contenedores2 = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2).next_to(codigo4.get_bottom())
        letra.move_to( contenedores2)
        grupo2 = Group(*[contenedores2, letra]).next_to(codigo4.get_right() + 2 * RIGHT) 
        
        contenedores3 = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2)
        palabra.move_to(contenedores3)
        grupo3 = Group(contenedores3, palabra)
        grupo3.shift(4 * RIGHT)

        self.play(FadeIn(grupo2), FadeIn(grupo3))
        self.play(grupo2.animate.move_to(DOWN * 2 + 1.2 * RIGHT), grupo3.animate.move_to(DOWN * 2 + 2.1 * RIGHT))
                  
        self.wait(3)
    

In [271]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):

        def crear_snippet(mi_codigo):
            """Crea una ventana de código y lo muestra palabra por palabra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/7.5
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):      
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace")
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ##### Memoria
        #################### Crear la grilla####################    
        cuadrados = VGroup(
            *[Square(side_length=0.6) for _ in range(72)]
            ).arrange_in_grid(rows=6, cols=12, buff = 0)  #, buff=MED_LARGE_BUFF)
        self.add(cuadrados)

        for cuadrado in cuadrados:
            cuadrado.set_color(BLACK)
            cuadrado.set_fill("#4B4B4B", opacity=1)#("#4D4D55", opacity =1)
            cuadrado.set_stroke(color=BLACK, width=1)
            
        circulos = VGroup(
            *[Circle(radius=0.2) for _ in range(72)]
        ).arrange_in_grid(rows=6, cols=12, buff=0.2)
        self.add(circulos)
        
        for circle in circulos:
            circle.set_color(BLACK)#("#DEDDDD")
            circle.set_fill(BLACK, opacity=1)#("#DEDDDD", opacity=1)

        grilla = VGroup(*[cuadrados, circulos])
        grilla.move_to(2.2 * DOWN + 3.5 * RIGHT)
        
        #####################################################
        # Crear pila desde grilla. Grilla consta de dos grupos: grilla[0] = cuadrados, grilla[1] = circulos
        primeros = [grilla[0][:12], grilla[1][:12]]
        pila = VGroup(*[primeros[0], primeros[1]])
        pila.move_to(2.3 * UP + 3.5 * RIGHT)
        # 
        self.play(Indicate(grilla[0][12]), Indicate(grilla[1][13]))
        

        self.wait(3)

        #### Fin memoria
        
        ## Animación
        

In [259]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):

        def crear_snippet(mi_codigo):
            """Crea una ventana de código y lo muestra palabra por palabra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/7.5
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):      
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace")
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        
        
        ## Animación
        explicacion = Text("Crear una lista con 5 elementos", font = "Arial Black").scale(0.7).shift(3.5*UP)
        mov = 1.65 * UP
        codigo = crear_codigo("""lista = [1, 2, 3, 4, 5]
print(lista)""", mov)
        texto_interno = Text("12345", font="Arial Black")
        # poner en escena la explicación
        self.play(Write(explicacion))
        # agregar ventana de código (solo la ventana)
        self.add(codigo[0])
        # Al crear la snippet se crea la animación de las líneas de código
        crear_snippet(codigo)
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8).move_to(2*DOWN + 1.5 * LEFT)
        # voy a hacer unos contenedores para tapar a los contenedores de las escena cuando necesite mostrar solo algunos elementos de la lista
        telones = crear_contenedores(texto_interno, "#24043d", "#24043d", 0.8,  0).move_to(2*DOWN + 1.5 * LEFT).shift(1*DOWN)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
        # Agregar los contenedores y el texto a la escena
        self.add(contenedores, texto_interno)
        
        # primer elemento
        explicacion2 = Text("Obtener primer elemento", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion2))
        mi_texto2 = """lista = [1, 2, 3, 4, 5]
print(lista[0])"""
  
        codigo2 = crear_codigo(mi_texto2, mov)
        self.add(codigo2[0])
        crear_snippet(codigo2)
        # Indicar
        for i in range(2):
            self.play(Circumscribe(contenedores[0]))

        self.play(telones[1].animate.move_to(contenedores[1]), telones[2].animate.move_to(contenedores[2]),
                  telones[3].animate.move_to(contenedores[3]), telones[4].animate.move_to(contenedores[4]))

        # último elemento
        explicacion3 = Text("Obtener el último elemento", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion3))
        mi_texto3 = """lista = [1, 2, 3, 4, 5]
print(lista[-1])"""
  
        codigo3 = crear_codigo(mi_texto3, mov)
        self.add(codigo3[0])
        crear_snippet(codigo3)
        # Indicar
        for i in range(2):
            self.play(Circumscribe(contenedores[-1]))

        self.play(telones[0].animate.move_to(contenedores[0]))
        self.play(telones[-1].animate.shift(2 * DOWN))
        # Añadir elemento suelto
        explicacion4 = Text("Agregar un elementos", font="Arial Black").scale(0.7).shift(3.5*UP)
        self.play(Transform(explicacion, explicacion3))
        self.play(FadeOut(telones))
        # append
        mi_texto4 = """lista = [1, 2, 3, 4, 5]
letra = 'a'
palabra = 'Python'
lista.append(letra)
lista.append(palabra)
print(lista)
"""
        letra = Text("'a'").scale(0.6)
        palabra = Text("'Python'").scale(0.3)
        mov = 1.65 * UP
        codigo4 = crear_codigo(mi_texto4, mov)
        self.add(codigo4[0])
        crear_snippet(codigo4)
        
        contenedores2 = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2).next_to(codigo4.get_bottom())
        letra.move_to( contenedores2)
        grupo2 = Group(*[contenedores2, letra]).next_to(codigo4.get_right() + 2 * RIGHT) 
        
        contenedores3 = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2)
        palabra.move_to(contenedores3)
        grupo3 = Group(contenedores3, palabra)
        grupo3.shift(4 * RIGHT)

        self.play(FadeIn(grupo2), FadeIn(grupo3))
        self.play(grupo2.animate.move_to(DOWN * 2 + 1.2 * RIGHT), grupo3.animate.move_to(DOWN * 2 + 2.1 * RIGHT))
        #self.play(contenedores3.animate.move_to(contenedores[-1]))
           
        self.wait(3)
    

In [162]:
%%manim -ql ListaAnimacion

class ListaAnimacion(ThreeDScene):
    def construct(self):
        logo = h.Logo()
        self.add(logo)
        self.set_camera_orientation(phi=45 * DEGREES, theta=255 * DEGREES, gamma=-5* DEGREES)
        
        # Crear el primer cubo
        primer_cubo = Cube(side_length=1.125).shift(4*LEFT + 1.5 * DOWN)
        
        # Lista para almacenar todos los cubos
        cubos = [primer_cubo]
        
        # Crear los 5 cubos restantes
        for _ in range(5):
            cubo = primer_cubo.copy().shift(1.4 * RIGHT)
            cubos.append(cubo)
            primer_cubo = cubo
        
        # Crear un grupo con todos los cubos
        grupo_cubos = VGroup(*cubos)
        
        # Añadir el grupo de cubos a la escena
        self.add(grupo_cubos)

        numero1 = Text("'hola'", color=WHITE).scale(0.55)
        numero1.move_to(grupo_cubos[0])
        self.add(numero1)
        grupo_cubos[2].shift(2*UP)

        # Mostrar la animación
        self.wait(2)


In [29]:
%%manim -ql ListaAnimacion
class ListaAnimacion(ThreeDScene):
    def construct(self):
    
        def crear_snippet(mi_codigo):
            """Recibe un objeto Code y lo dispone en pantalla letra por letra"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i])/8
                self.play(FadeIn(mi_codigo[1][i]))
                self.play(AddTextLetterByLetter(mi_codigo[2][i]), run_time=chars_por_seg, rate_func=linear)
            self.wait(1)

        def generar_snippet(mi_codigo):
            """Versión generador de crear_snippet"""
            for i in range(len(mi_codigo[2])):
                chars_por_seg = len(mi_codigo[2][i]) / 7.5
                # Animar otros objetos aquí, según sea necesario
                yield self.play(FadeIn(mi_codigo[1][i]))
                yield self.play(AddTextLetterByLetter(mi_codigo[2][i], run_time=chars_por_seg, rate_func=linear))
            self.wait(1)

        def crear_contenedores(un_texto, color_trazo=WHITE, color=BLUE, lado=1.2, trazo=2):
            """Crea una secuencia de figuras que sirven de contenedores para un texto.
            Hacemos uso de arrange_in_grid para que el texto quede contenido en cada una de las figuras."""
            cuadrados = VGroup(*[Square(side_length=lado,
                                color=color_trazo,
                                fill_color=color,
                                fill_opacity=1,
                                stroke_width=trazo) for i in range(len(un_texto))])
            cuadrados.arrange_in_grid(rows=1, buff=0.1)
            return cuadrados
         
        def alinear_elementos(un_texto, contenedores):
            # Alinear horizontalmente las letras con la parte inferior de los cuadrados
            for letra, cuadrado in zip(un_texto, contenedores):
                letra.move_to(cuadrado.get_bottom(), aligned_edge=DOWN)
            un_texto.shift(0.2 * UP)

        def crear_codigo(un_texto, movimiento):
            codigo = Code(code = un_texto, tab_width = 2.25, background = "window",language = "Python", font="Monospace").scale(0.85)
            codigo.move_to(movimiento)
            return codigo
        # Obtener el tamaño de la pantalla
        ancho = self.camera.frame_width
        alto = self.camera.frame_height

        # Añadir logo
        logo = h.Logo()
        self.add(logo)

        grilla = h.GrillaRegla()
        self.add(grilla)

        ## Animación
        lineas_codigo = """lista = [1, 2, 3, 4, 5]
letra  = 'a'
palabra = 'Python'
lista2 = ["otra", "lista"]
lista.append(a)
lista.append(palabra)
lista1.append(lista2)
"""

        explicacion = Text("Crear una lista con 5 elementos", font = "Arial Black").scale(0.7).shift(3.5*UP)
        resultado = Text("Resultado:", font = "Arial Black").scale(0.6).shift(3.5*RIGHT + 2.25 * UP)
        mov = 1.25 * UP + 4.25 * LEFT

        codigo = crear_codigo(lineas_codigo, mov)
        texto_interno = Text("12345", font="Arial Black")
        texto_interno2 = Text('["otra", "lista"]', font="Arial Black").scale(0.125)
        # poner en escena la explicación
        self.play(Write(explicacion))
        # Añado la ventana de código (solo la ventana), luego creo el contenido. Es una forma de desdoblar la animación del objeto Code().
        self.add(codigo[0])
        # Al crear la snippet se crea la animación de las líneas de código
        # texto resultado
        self.play(Write(resultado))
        # Crear un generador con mi código. Existen dos objetos dentro de cada línea de Code(). Primero el número, después la línea
        gen = generar_snippet(codigo)
        # crear los contenedores de los elementos de la lista     
        contenedores = crear_contenedores(texto_interno, WHITE, BLUE, 0.8)
        #contenedores2 = crear_contenedores(texto_interno2, WHITE, BLUE, 0.8)
        # Alinear horizontalmente las letras con la parte inferior de los cuadrados
        alinear_elementos(texto_interno, contenedores)
       # alinear_elementos(texto_interno2, contenedores2)
        # una lista que queda tras de escenas
        lista = Group(*[contenedores, texto_interno])
        lista_copia = lista.copy().move_to(3.25 * RIGHT + 1.5 * UP)
        #lista_copia = lista.copy().move_to(1.75 * LEFT) # + 2.5 * DOWN)
       
        cuad_letra = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2) 
        letra = Text("'a'").scale(0.75)
        letra.move_to(cuad_letra)
        letra_grupo = Group(*[cuad_letra, letra])
        letra_grupo.move_to(lista_copia[0][0]).shift(0.9*DOWN)

        cuad_palabra = Square(side_length=0.8, color=WHITE, fill_color=GREEN, fill_opacity=1, stroke_width=2)
        palabra = Text("'Python'").scale(0.3)
        palabra.move_to(cuad_palabra)
        palabra_grupo = Group(*[cuad_palabra, palabra])
        palabra_grupo.move_to(letra_grupo).shift(0.9*DOWN)

        cuad_lista2 = Square(side_length=0.8, color=WHITE, fill_color=ORANGE, fill_opacity=1, stroke_width=2)
        texto_interno2.move_to(cuad_lista2)
        lista2 = Group(*[cuad_lista2, texto_interno2])
        lista2.move_to(palabra_grupo).shift(0.9 * DOWN)
        #lista2_copia = lista.copy().move_to(palabra_grupo).shift(0.9*DOWN)
         
        # Animar la primera línea de Code usando el generador. 
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        pointer = Arrow(codigo[2][0].get_right() + 1.25 * RIGHT, codigo[2][0].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        self.play(GrowArrow(pointer))

        # animación de lista, mostrándose en escena
        self.play(FadeIn(lista_copia))
          
        # animo la segunda línea de Code()
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer2 = Arrow(codigo[2][1].get_right() + 1.25 * RIGHT, codigo[2][1].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer2))
        #
        self.play(FadeIn(letra_grupo))
        
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer3 = Arrow(codigo[2][2].get_right() + 1.25 * RIGHT, codigo[2][2].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer3))

        self.play(FadeIn(palabra_grupo))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer4 = Arrow(codigo[2][3].get_right() + 1.25 * RIGHT, codigo[2][3].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer4))
        #
        self.play(FadeIn(lista2))
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer5 = Arrow(codigo[2][4].get_right() + 1.25 * RIGHT, codigo[2][4].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer5))
        
        self.play(lista_copia.animate.move_to(1.75 * LEFT + 2.5 * DOWN))
        self.play(letra_grupo.animate.next_to(lista_copia[0][-1].get_center(), RIGHT * 2))
        
        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer6 = Arrow(codigo[2][5].get_right() + 1.25 * RIGHT, codigo[2][5].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer6))

        self.play(palabra_grupo.animate.next_to(letra_grupo.get_center(), RIGHT * 2))

        for i in range(2):
            valor = next(gen) #  Obtener valores del generador, el primero es el número de fila, luego viene la línea
            # no hace falta la notación "self."" porque ya está hecho en el generador
            valor
        # otro pointer para una nueva línea de código
        pointer6 = Arrow(codigo[2][6].get_right() + 1.25 * RIGHT, codigo[2][6].get_right() + 0.15 * LEFT,
                        max_stroke_width_to_length_ratio=9, stroke_width=9, color=GREEN)
        # pasar a siguiente línea
        self.play(Transform(pointer, pointer6))
        self.play(lista2.animate.next_to(palabra_grupo.get_center(), RIGHT * 2))
          
        self.wait(3)

In [21]:
lista = [1,2,3,4,5,6,7]
lista.remove(7)
lista

[1, 2, 3, 4, 5, 6]

In [22]:
del lista[-1]

In [23]:
lista

[1, 2, 3, 4, 5]